### Сщздание ONNX "модели", выполняющей размытие изображения по маске.

In [1100]:
from PIL import Image
import json
from typing import Tuple
import math

from glob import glob
from tqdm.notebook import tqdm
from pathlib import Path

import numpy as np
from openvino.runtime import Core

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
sns.set(style='darkgrid', font_scale=1.2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)
np.random.seed(0)

cuda


Загружаем изображение для визуализации:

In [949]:
img_path = "/home/maksim/Downloads/1440355628_gavajskij-koktejl-potok-lavy.jpg"

In [ ]:
Image.open(img_path)

In [1426]:
img = Image.open(img_path).convert("RGB")
img = img.resize((img.size[0]//2*2, img.size[1]//2*2))
image = np.asarray(img) / 255
image = torch.tensor(np.moveaxis(image, 2, 0), dtype=torch.float32)[None, ...]
image.shape

torch.Size([1, 3, 820, 616])

Функция для расчета маски размытия по координатам bounding box

In [1427]:
def __generate_blur_mask(size: Tuple[int, int], b_box: Tuple[float, float, float, float]) -> np.array:
    height, width = size
    y_min, x_min, y_max, x_max = b_box

    x_min, x_max = round(x_min * width), round(x_max * width)
    y_min, y_max = round(y_min * height), round(y_max * height)

    result = np.zeros((width, height), dtype=np.float32)

    x = np.linspace(x_min/width, 0, x_min)
    y = np.linspace(y_min/height, 0, y_min)
    yv, xv = np.meshgrid(y, x)
    box1 = (1 / ((xv ** 2 + yv ** 2) ** 0.5 + 1)).astype(np.float32)

    box2 = 1 / (np.full((x_max - x_min, y_min), np.linspace(y_min/height, 0, y_min)) + 1)

    x = np.linspace(0, (width - x_max)/width, width - x_max)
    y = np.linspace(y_min/height, 0, y_min)
    yv, xv = np.meshgrid(y, x)
    box3 = (1 / ((xv ** 2 + yv ** 2) ** 0.5 + 1)).astype(np.float32)

    box4 = 1 / (np.full((y_max - y_min, width - x_max), np.linspace(0, (width - x_max)/width, width - x_max)) + 1).T

    x = np.linspace(0, (width - x_max)/width, width - x_max)
    y = np.linspace(0, (height - y_max)/height, height - y_max)
    yv, xv = np.meshgrid(y, x)
    box5 = (1 / ((xv ** 2 + yv ** 2) ** 0.5 + 1)).astype(np.float32)

    box6 = 1 / (np.full((x_max - x_min, height - y_max), np.linspace(0, (height - y_max)/height, height - y_max)) + 1)

    x = np.linspace(x_min/width, 0, x_min)
    y = np.linspace(0, (height - y_max)/height, height - y_max)
    yv, xv = np.meshgrid(y, x)
    box7 = (1 / ((xv ** 2 + yv ** 2) ** 0.5 + 1)).astype(np.float32)

    box8 = 1 / (np.full((y_max - y_min, x_min), np.linspace(x_min/width, 0, x_min)) + 1).T

    result[x_min: x_max, y_min: y_max] = 1.0

    result[0: x_min, 0: y_min] = box1
    result[x_min: x_max, 0: y_min] = box2
    result[x_max:, 0: y_min] = box3
    result[x_max:, y_min: y_max] = box4
    result[x_max:, y_max:] = box5
    result[x_min: x_max, y_max:] = box6
    result[0: x_min, y_max:] = box7
    result[0: x_min, y_min: y_max] = box8
    

    return result[None, None, :, :]**2


Pytorch "Модель". Содержит свертку с Гауссовым ядром, выполняющую размывание, свертки для уменьшения и увеличения размера изображения для ускорения работы, наложение размытых изображений и исходного изображения по маске.

In [1450]:
class Blur(nn.Module):
    def __init__(self, kernel_size):
        super(Blur, self).__init__()
        self.blur = nn.Conv2d(3, 3, kernel_size=kernel_size, stride=1, padding=kernel_size//2, bias=False, padding_mode='replicate')
        self.zip = nn.Conv2d(4, 4, kernel_size=2, stride=2, bias=False)
        self.unzip =  nn.ConvTranspose2d(3, 3, kernel_size=2, stride=2, bias=False)
        
        # Gaussian blur kernels
        k0 = torch.zeros((1, 1, kernel_size, kernel_size), dtype=torch.float32)
        k1 = torch.tensor(self.calc_gaussian_kernel(kernel_size, kernel_size*1.25), dtype=torch.float32)[None, None, :, :]
        kernel = torch.cat((torch.cat((k1, k0, k0), dim=1),
                 torch.cat((k0, k1, k0), dim=1),
                 torch.cat((k0, k0, k1), dim=1)))
        for i in range(kernel.shape[0]):
            self.state_dict()['blur.weight'][i] = kernel[i]

        # Zip kernels
        k0 = torch.zeros((1, 1, 2, 2), dtype=torch.float32)
        k1 = torch.ones((1, 1, 2, 2), dtype=torch.float32) * 0.25
        kernel = torch.cat((torch.cat((k1, k0, k0, k0), dim=1),
                 torch.cat((k0, k1, k0, k0), dim=1),
                 torch.cat((k0, k0, k1, k0), dim=1),
                 torch.cat((k0, k0, k0, k1), dim=1)))
        for i in range(kernel.shape[0]):
            self.state_dict()['zip.weight'][i] = kernel[i]
            
        # Unzip cernels
        k0 = torch.zeros((1, 1, 2, 2), dtype=torch.float32)
        k1 = torch.ones((1, 1, 2, 2), dtype=torch.float32)
        kernel = torch.cat((torch.cat((k1, k0, k0), dim=1),
                 torch.cat((k0, k1, k0), dim=1),
                 torch.cat((k0, k0, k1), dim=1)))
        for i in range(kernel.shape[0]):
            self.state_dict()['unzip.weight'][i] = kernel[i]
            
        
    def calc_gaussian_kernel(self, size: int, fwhm: float) -> np.array:
        """ Make a square gaussian kernel.

        size is the length of a side of the square
        fwhm is full-width-half-maximum, which
        can be thought of as an effective radius.
        """
        x = np.arange(0, size, 1, float)
        y = x[:,np.newaxis]
        x0 = y0 = size // 2
        kernel = np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)
        return kernel / kernel.sum()
        
    def forward(self, x):
        initial = x[:, :3, :, :]
        mask = x[:, 3:4, :, :]
        x_zip = self.zip(x)
        blured_zip = x_zip[:, :3, :, :]
        initial_zip = x_zip[:, :3, :, :]
        mask_zip = x_zip[:, 3:4, :, :]
        
        blured_1_zip = self.blur(blured_zip)
        
        blured_2_zip = blured_1_zip * (1-mask_zip) + initial_zip * mask_zip
        
        blured_2_zip = self.blur(blured_2_zip)
            
        blured_3_zip = blured_2_zip * (1-mask_zip**3) + initial_zip * mask_zip**3
        
        for i in range(2):
            blured_3_zip = self.blur(blured_3_zip)
            
        blured_4 = self.unzip(blured_3_zip) * (1-mask**9) + initial * mask**9

        return blured_4
    

In [1459]:
KERNEL_SIZE = 17
model = Blur(KERNEL_SIZE).to('cpu')

Проверяем работу:

In [1461]:
mask = __generate_blur_mask((image.shape[3], image.shape[2]), (0.35, 0.4, 0.7, 0.95))
blured_image = model(torch.tensor(np.concatenate((image, mask), axis=1))).detach().numpy()
print(mask.min())

0.42610756


#### Конвертация в ONNX

In [1463]:
# Evaluate the model to switch some operations from training mode to inference.
model.eval()
# Create dummy input for the model. It will be used to run the model inside export function.
dummy_input = torch.randn(1, 4, 320, 320)
# Call the export function
torch.onnx.export(model,
                  args=dummy_input,
                  f='blur.onnx',
                  export_params=True,
                  opset_version=11,
                  do_constant_folding=True,
                  input_names = ['image'],
                  output_names = ['output'],
                  dynamic_axes={'image' : {2: 'width', 3: 'height'},
                                'output' : {2: 'width', 3: 'height'}})


Проверяем инференс на OpenVino

In [1464]:
ie = Core()
onnx_model_path = 'blur.onnx'
model_onnx = ie.read_model(model=onnx_model_path)
compiled_model_onnx = ie.compile_model(model=model_onnx, device_name="CPU")

input_layer = compiled_model_onnx.input(0)
output_layer = compiled_model_onnx.output(0)
infer_request = compiled_model_onnx.create_infer_request()

In [1465]:
input = np.concatenate((image, mask), axis=1)
blured_image = infer_request.infer([input])[output_layer][0]